## Core Workflow: Get NAIP imageries from given addresses
Purpose: The two primary prediction study area are Los Angeles county and Kansas City. For LA, the official building footprint data from LA city government are used to get the geometry of the rooftops. For Kansas City, the rooftop geometries are acquired from Microsoft footprint data. The geometries are then used to acquire NAIP imagery for all the roofs within the AOI. The mean band values (R, G, B, NIR) for each rooftop are saved and used for subsequent operation. 
<br>
*Date: 2019-10-31*



### Import statements

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#
import os
import sys
import json
import itertools
import pickle
from pprint import pprint
#
import numpy as np
import shapely
from shapely.geometry import shape, Point
from shapely.geometry import mapping, Polygon
# import cartopy
import geojson
import fiona
import gdal
import h5py
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler 
# import ogr, gdal
from glob import glob

import requests
import logging
import time

import pandas as pd

import collections

import rasterio as rio
from rasterio.plot import show
from numpy import mean

import random
import statistics

import descarteslabs as dl
from descarteslabs.vectors import FeatureCollection

print (sys.path)

### Set input variables

In [ ]:
# set the AOI for processing (LA for Los Angeles or KC for Kansas City)
prediction_city = 'LA'

# set the year to process 
prediction_year = 2009

# range of date to search for imagery
start_date = '2009-01-01'
end_date = '2009-12-01'

# resolution of the downloaded imagery
resolution = 1

### Function for multiprocessing

In [ ]:
import itertools
from multiprocessing import Process, cpu_count
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool
from datetime import datetime


#
# CONFIG
#
MAX_POOL_PROCESSES=cpu_count()-1
MAX_THREADPOOL_PROCESSES=16



#
# METHODS
#
""" MAP METHODS
  Args:
    * map_function <function>: 
      a function to map over args list. the function should take a single argument.
      if multiple arguments are needed accept them as a single list or tuple
    * args_list <list>: the list of arguments to map over
    * max_process <int>: number of processes
      - for max_with_pool defaults to the number of cpus minus 1
      - for max_with_threadpool defaults to 16
      - map_sequential ignores this argument as its doesn't actually do 
        any multiprocesssing 
  Return:
    List of return values from map_function
  Notes:
    map_sequential does NOT multiprocess.  it can be used as a sequential drop-in 
    replacement for map_with_pool/threadpool.  this is useful for:
      - development 
      - debugging
      - benchmarking 
"""
def map_with_pool(map_function,args_list,max_processes=MAX_POOL_PROCESSES):
  pool=Pool(processes=min(len(args_list),max_processes))
  return _run_pool(pool,map_function,args_list)


def map_with_threadpool(map_function,args_list,max_processes=MAX_THREADPOOL_PROCESSES):
  pool=ThreadPool(processes=min(len(args_list),max_processes))
  return _run_pool(pool,map_function,args_list)


def map_sequential(map_function,args_list,print_args=False,noisy=False,**dummy_kwargs):
  if noisy:
    print('multiprocessing(test):')
  out=[]
  for i,args in enumerate(args_list):
      if noisy: 
        print('\t{}...'.format(i))
      if print_args:
        print('\t{}'.format(args))
      out.append(map_function(args))
  if noisy: 
    print('-'*25)
  return out





""" simple: vanilla multiprocessing
  Args:
    * function <function>: function. function can take multiple arguments 
    * args_list <list>: the list of argument lists
    * join <bool[True]>: join processes before return
  Return: 
    List of processes 
"""
def simple(function,args_list,join=True):
  procs=[]
  for args in args_list:
      proc=Process(
          target=function, 
          args=args)
      procs.append(proc)
      proc.start()
  if join:
    for proc in procs:
        proc.join()
  return procs






""" MPList
Run the above methods on map_function,args_list pairs where the map_function
changes for each new set of args in args_list
Args:
    pool_type<str>: 
        one of MPList.POOL|THREAD|SEQUENTIAL.  determines which map_function 
        and default max_processes to use. If not MPList.THREAD|SEQUENTIAL it 
        will default to MPList.POOL.
    max_processes<int>:
        if not passed will set default based on pool_type
    jobs<list>:
        list of (target,args,kwargs) tuples. Note: use the append method rather than
        creating (target,args,kwargs) tuples
        
"""
class MPList():
    #
    # POOL TYPES
    #
    POOL='pool'
    THREAD='threading'
    SEQUENTIAL='sequential'
    

    #
    # PUBLIC
    #
    def __init__(self,pool_type=None,max_processes=None,jobs=None):
        self.pool_type=pool_type or self.POOL
        self.max_processes=max_processes
        self.jobs=jobs or []

        
    def append(self,target,*args,**kwargs):
        self.jobs.append((target,)+(args,)+(kwargs,))
        
    
    def run(self):
        self.start_time=datetime.now()
        map_func,self.max_processes=self._map_func_max_processes()
        out=map_func(self._target,self.jobs,max_processes=self.max_processes)
        self.end_time=datetime.now()
        self.duration=str(self.end_time-self.start_time)
        return out
        

    def __len__(self):
        return len(self.jobs)
    
    
    #
    # INTERNAL
    #    
    def _map_func_max_processes(self):
        if self.pool_type==MPList.THREAD:
            map_func=map_with_threadpool
            max_processes=self.max_processes or MAX_THREADPOOL_PROCESSES
        elif self.pool_type==MPList.SEQUENTIAL:
            map_func=map_sequential
            max_processes=False
        else:
            map_func=map_with_pool
            max_processes=self.max_processes or MAX_POOL_PROCESSES
        return map_func, max_processes
        
        
    def _target(self,args):
        target,args,kwargs=args
        return target(*args,**kwargs)
        
    

#
# INTERNAL METHODS
#
def _stop_pool(pool,success=True):
  pool.close()
  pool.join()
  return success


def _map_async(pool,map_func,objects):
  try:
    return pool.map_async(map_func,objects)
  except KeyboardInterrupt:
    print("Caught KeyboardInterrupt, terminating workers")
    pool.terminate()
    return False
  else:
    print("Failure")
    return _stop_pool(pool,False)


def _run_pool(pool,map_function,args_list):
  out=_map_async(pool,map_function,args_list)
  _stop_pool(pool)
  return out.get()

In [ ]:
def arg_dict_decorator(func):
    def decorator(arg_dict):
        return func(**arg_dict)
    return decorator


@arg_dict_decorator
def calc_bands_LA_2012(type,properties,geometry):    
    attempts = 0

    while attempts < 2:
        try:
            cnt = properties['cnt']
            polygon = shape(geometry)
            bld_id = properties['BLD_ID']
            shp_area = properties['Shape_Area']
            
            # search for scenes from Descartes lab platform
            scenes, ctx = dl.scenes.search(geometry, products=product, start_datetime=start_date, 
                                           end_datetime=end_date, limit=None)

            rf_no = cnt

            img_id = - 1    

            for scene in scenes:

                img_id = img_id + 1
                
                # save the band values as arrays
                naip_data = scene.ndarray(bands="red green blue nir", ctx=ctx.assign(resolution=resolution),mask_alpha=False)
                red = naip_data[0]
                red = red.astype(float)

                green = naip_data[1]
                green = green.astype(float)

                blue = naip_data[2]
                blue = blue.astype(float)
                
                nir = naip_data[3]
                nir = nir.astype(float)
                        
                arr = [red,green,blue,nir]

                flat_arr = []
                # flattened array of tuples
                flat_list = zip(*map(lambda x:x.flatten(),arr))
                for i in flat_list:
                    flat_arr.append(i)   
                    
                selected_pixels=[]
                # remove blank pixels 
                for pixels in flat_arr:
                    if pixels[0] != 0 or pixels[1] != 0 or pixels[2] != 0 or pixels[3] != 0:
                        selected_pixels.append(pixels)
                        
                # raw band values        
                raw_red_b = []
                raw_green_b = []
                raw_blue_b = []
                raw_nir_b = []

                for pixels in selected_pixels:
                    raw_red_b.append(pixels[0]) 
                    raw_green_b.append(pixels[1])
                    raw_blue_b.append(pixels[2])
                    raw_nir_b.append(pixels[3])
                    
                # calculate the raw mean values for all the bands from this list
                raw_red_mean=mean(raw_red_b)
                raw_green_mean=mean(raw_green_b)
                raw_blue_mean=mean(raw_blue_b)
                raw_nir_mean=mean(raw_nir_b)

                total_pixel = len(selected_pixels) # calculate the number of pixels within the roof used for calculation               

                imgs.append(img_id)
                roofs.append(rf_no)
                footprint_shapes.append(polygon)

                total_pixels.append(total_pixel)            

                raw_reds.append(raw_red_mean)
                raw_greens.append(raw_green_mean)
                raw_blues.append(raw_blue_mean)
                raw_nirs.append(raw_nir_mean)
                
                bld_ids.append(bld_id)
                shp_areas.append(shp_area)

            break
        except Exception as e:
            attempts += 1
            if attempts == 2:
                print('unsuccessfull at count', cnt)
                unsuc_cnts.append(cnt)
            else:
                time.sleep(2)


In [ ]:
def arg_dict_decorator(func):
    def decorator(arg_dict):
        return func(**arg_dict)
    return decorator


@arg_dict_decorator
def calc_bands_LA_2014(type,properties,geometry):    
    attempts = 0

    while attempts < 2:
        try:
            cnt = properties['cnt']
            polygon = shape(geometry)
            
            if properties['OLD_BLD_ID'] == None:
                bld_id = properties['BLD_ID']
            else:
                bld_id = properties['OLD_BLD_ID']
                
            status = properties['STATUS']
            use_type = properties['UseType']
            
            shp_area = properties['Shape_Ar_1']

            scenes, ctx = dl.scenes.search(geometry, products=product, start_datetime=start_date, 
                                           end_datetime=end_date, limit=None)

            rf_no = cnt

            img_id = - 1    

            for scene in scenes:

                img_id = img_id + 1

                naip_data = scene.ndarray(bands="red green blue nir", ctx=ctx.assign(resolution=resolution),mask_alpha=False)
                red = naip_data[0]
                red = red.astype(float)

                green = naip_data[1]
                green = green.astype(float)

                blue = naip_data[2]
                blue = blue.astype(float)
                
                nir = naip_data[3]
                nir = nir.astype(float)
                        
                arr = [red,green,blue,nir]

                flat_arr = []
                # flattened array of tuples
                flat_list = zip(*map(lambda x:x.flatten(),arr))
                for i in flat_list:
                    flat_arr.append(i)   
                    
                selected_pixels=[]
                # remove blank pixels 
                for pixels in flat_arr:
                    if pixels[0] != 0 or pixels[1] != 0 or pixels[2] != 0 or pixels[3] != 0:
                        selected_pixels.append(pixels)
                        
                # raw band values        
                raw_red_b = []
                raw_green_b = []
                raw_blue_b = []
                raw_nir_b = []

                for pixels in selected_pixels:
                    raw_red_b.append(pixels[0]) 
                    raw_green_b.append(pixels[1])
                    raw_blue_b.append(pixels[2])
                    raw_nir_b.append(pixels[3])
                    
                # calculate the raw mean values for all the bands from this list
                raw_red_mean=mean(raw_red_b)
                raw_green_mean=mean(raw_green_b)
                raw_blue_mean=mean(raw_blue_b)
                raw_nir_mean=mean(raw_nir_b)

                total_pixel = len(selected_pixels) # calculate the number of pixels within the roof used for calculation               

                imgs.append(img_id)
                roofs.append(rf_no)
                footprint_shapes.append(polygon)

                total_pixels.append(total_pixel)             

                raw_reds.append(raw_red_mean)
                raw_greens.append(raw_green_mean)
                raw_blues.append(raw_blue_mean)
                raw_nirs.append(raw_nir_mean)
                
                bld_ids.append(bld_id)
                shp_areas.append(shp_area)
                statuss.append(status)
                use_types.append(use_type)

            break
        except Exception as e:
            attempts += 1
            if attempts == 2:
                print('unsuccessfull at count', cnt)
                unsuc_cnts.append(cnt)
            else:
                time.sleep(2)


In [ ]:
def arg_dict_decorator(func):
    def decorator(arg_dict):
        return func(**arg_dict)
    return decorator


@arg_dict_decorator
def calc_bands_KC(type,properties,geometry):    
    attempts = 0

    while attempts < 2:
        cnt = (properties['cnt'])
        rf_no = cnt
        img_id = - 1

        try:
            polygon = shape(geometry)

            scenes, ctx = dl.scenes.search(geometry, products=product, start_datetime=start_date, 
                                           end_datetime=end_date, limit=None)                            

            for scene in scenes:

                # find the state that the imagery belong to
                state = scene.properties.directory
                state = state[0:2]

                img_id = img_id + 1

                naip_data = scene.ndarray(bands="red green blue nir", ctx=ctx.assign(resolution=1),mask_alpha=False)
                red = naip_data[0]
                red = red.astype(float)

                green = naip_data[1]
                green = green.astype(float)

                blue = naip_data[2]
                blue = blue.astype(float)

                nir = naip_data[3]
                nir = nir.astype(float)

                arr = [red,green,blue,nir]

                flat_arr = []
                # flattened array of tuples
                flat_list = zip(*map(lambda x:x.flatten(),arr))
                for i in flat_list:
                    flat_arr.append(i)   

                selected_pixels=[]
                # remove blank pixels and normalize for scenes
                for pixels in flat_arr:
                    if pixels[0] != 0 or pixels[1] != 0 or pixels[2] != 0 or pixels[3] != 0:
                        selected_pixels.append(pixels)

                # raw band values        
                raw_red_b = []
                raw_green_b = []
                raw_blue_b = []
                raw_nir_b = []

                for pixels in selected_pixels:
                    raw_red_b.append(pixels[0]) 
                    raw_green_b.append(pixels[1])
                    raw_blue_b.append(pixels[2])
                    raw_nir_b.append(pixels[3])

                # calculate the mean values for all the bands from this list
                raw_red_mean=mean(raw_red_b)
                raw_green_mean=mean(raw_green_b)
                raw_blue_mean=mean(raw_blue_b)
                raw_nir_mean=mean(raw_nir_b)

                total_pixel = len(selected_pixels) # calculate the size of the roof               

                imgs.append(img_id)
                roofs.append(rf_no)
                footprint_shapes.append(polygon)

                total_pixels.append(total_pixel)              

                raw_reds.append(raw_red_mean)
                raw_greens.append(raw_green_mean)
                raw_blues.append(raw_blue_mean)
                raw_nirs.append(raw_nir_mean)

                states.append(state)


            break
        except Exception as e:
            attempts += 1
            if attempts == 2:
                print('unsuccessfull at count', cnt)
                unsuc_cnts.append(rf_no)
            else:
                time.sleep(2)

### Load the current footprint source based on prediction year

In [ ]:
if prediction_city == 'LA':
    if prediction_year <= 2012:
        with open('/data/phase_i/official_footprints/LA_county_official_footprints_2014.geojson') as f:
            js = json.load(f)
    else:
        with open('/data/phase_i/official_footprints/LA_county_official_footprints_2009.geojson') as f:
            js = json.load(f)
elif prediction_city == 'KC':
    with open('/data/phase_i/microsoft_footprints/kansas_city_MS_building_footprints.geojson') as f:
        js = json.load(f)

In [ ]:
arg_list = js['features']

print(len(arg_list))

cnt = -1

# Add a unique id to each roofs
for feat in arg_list:
    cnt = cnt + 1
    feat['properties']['cnt'] = cnt

### Run the main function to process the data 

In [ ]:
product = u'usda:naip:rgbn:v1'

roofs = []
imgs = []
footprint_shapes=[]
total_pixels = []

raw_reds = []
raw_greens = []
raw_blues = []
raw_nirs = []

unsuc_cnts = []

bld_ids = []
shp_areas = []
statuss = []
use_types = []

states = []

if prediction_city == 'LA':
    if prediction_year <= 2012:
        %time out = map_with_threadpool(calc_bands_LA_2012,arg_list,max_processes=32)
    else:     
        %time out = map_with_threadpool(calc_bands_LA_2014,arg_list,max_processes=32)

elif prediction_city == 'KC':
    %time out = map_with_threadpool(calc_bands_KC,arg_list,max_processes=32)

In [ ]:
print('finished multiprocessing')

print('unsuccessfull counts = ', len(unsuc_cnts))

### Store the results to a pandas library and then save to a csv file.

In [ ]:
if prediction_city == 'LA':
    if prediction_year <= 2012:
        # store the results to a pandas library.
        df = pd.DataFrame({ 'roof_no': roofs, 'img_id':imgs, 'footprint_shapes':footprint_shapes,'total_pixels': total_pixels,
                           'bld_id':bld_ids, 'shp_area':shp_areas, 'status':statuss, 'use_type':use_types,
                          'raw_red_mean':raw_reds,'raw_green_mean': raw_greens,'raw_blue_mean': raw_blues,'raw_nir_mean': raw_nirs})
        # Write the full results to csv using the pandas library. 
        df.to_csv('band_values_NAIP_LA_'+prediction_year+'.csv',encoding='utf8')
    else:
        # store the results to a pandas library.
        df = pd.DataFrame({ 'roof_no': roofs, 'img_id':imgs, 'footprint_shapes':footprint_shapes,'total_pixels': total_pixels,
                           'bld_id':bld_ids, 'shp_area':shp_areas,
                    'raw_red_mean':raw_reds,'raw_green_mean': raw_greens,'raw_blue_mean': raw_blues,'raw_nir_mean': raw_nirs})

        # Write the full results to csv using the pandas library. 
        df.to_csv('band_values_NAIP_LA_'+prediction_year+'.csv',encoding='utf8')

elif prediction_city == 'KC':
    # store the results to a pandas library.
    df = pd.DataFrame({ 'roof_no': roofs, 'img_id':imgs, 'footprint_shapes':footprint_shapes,'total_pixels': total_pixels,
                       'state': states,
                      'raw_red_mean':raw_reds,'raw_green_mean': raw_greens,'raw_blue_mean': raw_blues,'raw_nir_mean': raw_nirs})

    # Write the full results to csv using the pandas library. 
    df.to_csv('band_values_NAIP_KC_'+prediction_year+'.csv',encoding='utf8')

### Inspect the dataframe

In [ ]:
df

----------------------------------------